# RAG Pipeline for Smart Contract Generation

This notebook demonstrates the complete workflow of converting legal documents into smart contracts using RAG (Retrieval Augmented Generation) and LLM.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
with open('../docs/short-term-vacation-lease-agreements/agreement_1.txt', 'r') as file:
    rental_agreement = file.read()

In [ ]:
# Prepare the combined prompt
prompt = """
Create a Solidity smart contract for a rental agreement system with the following requirements:

1. Contract Structure:
- State variables for lessor and lessee addresses
- Property details (address hash)
- Rent amount in ETH
- Agreement duration and end date
- Payment tracking

2. Core Functionality:
- Function to initialize new rental agreements
- Rent payment processing
- Agreement termination handling

3. Rights and Obligations:
- Lessor's right to receive rent
- Lessee's obligation to pay on time
- Property maintenance responsibilities
- Inspection rights with notice
- No sublease without consent

4. Security Features:
- Secure payment handling
- Access control for functions
- Event logging for all actions

5. Dispute Resolution:
- Arbitration mechanism
- Breach handling
- Early termination conditions

6. Additional Requirements:
- Include proper error handling
- Emit events for important state changes
- Follow Solidity best practices and security patterns

As a hint use the following context , which consists of a text description from a part of similar legal agreement and its corresponding Solidity code template (separated by '--- Solidity Template №xxx ---'), to fulfill the user's request:
{context}

Convert this rental agreement to Solidity smart contract:
{agreement_to_convert}

Solidity contract:
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

// Begin implementation below:
"""

In [ ]:
from langchain_community.llms import LlamaCpp
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

import rag


model = LlamaCpp(
    model_path="../models/codellama-7b-instruct.Q5_K_M.gguf",
    n_gpu_layers=-1,
    n_batch=16,
    n_ctx=8192,
    temperature=0.75,
    max_tokens=2500,
    top_p=1
)
templates = rag.load_templates("../templates/")
embeddings = SentenceTransformerEmbeddings(
    model_name="all-MiniLM-L6-v2", model_kwargs={'device': 'cuda'})
vector_store = rag.create_vector_store(embeddings, templates)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200)
rag_chain = rag.create_chain(vector_store, model, text_splitter, prompt)

In [ ]:
model.invoke("generate hello world program on java")

In [ ]:
from IPython.display import Image, display

display(Image(rag_chain.get_graph().draw_mermaid_png()))

In [ ]:
agreement_part = """
  4.  SECURITY DEPOSIT. The Tenant shall be obligated to pay the following amounts upon the execution of this Agreement: (check one)

  ☑ - Security Deposit: $3000 (“Security Deposit”). The Security Deposit is for the faithful performance of the Tenant under the terms and conditions of this Agreement. The Tenant must pay the Security Deposit at the execution of this Agreement. The Security Deposit shall be returned to the Tenant within the State's requirements after the end of the Lease Term less any itemized deductions. This Security Deposit shall not be credited towards any Rent unless the Landlord gives their written consent.
"""

print(vector_store.similarity_search(agreement_part)[0].page_content)

In [ ]:
print("Invoking RAG chain...")
response = rag_chain.invoke({"agreement_to_convert": rental_agreement})
print("Response is generated")

In [ ]:
print(response["answer"])